In [ ]:
#! /usr/bin/python3.5
#https://learningtensorflow.com/ReadingFilesBasic/
#https://www.tensorflow.org/tutorials/using_gpu

In [85]:
#To compute term frequancy from multi txt file and save it in multi csv and one csv by parellel process using tensorflow on GPU


In [2]:
import os

import tensorflow as tf

'''import csv
from tensorflow.python.client import timeline
import nltk
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
from collections import Counter
import threading

import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import PorterStemmer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from string import digits'''
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



'import csv\nfrom tensorflow.python.client import timeline\nimport nltk\nfrom nltk.wsd import lesk\nfrom nltk.corpus import wordnet as wn\nfrom collections import Counter\nimport threading\n\nimport pandas as pd\nimport numpy as np\nfrom nltk.tokenize import sent_tokenize, word_tokenize\n\nfrom nltk.corpus import stopwords\nfrom nltk.tokenize import RegexpTokenizer\nfrom nltk import PorterStemmer\nfrom nltk.stem.porter import *\nfrom nltk.stem.snowball import SnowballStemmer\nfrom string import digits'

# Configuration Data

## Pathes

### Data Source

In [3]:
dir_path = os.path.dirname(os.path.realpath(__file__))

path_data_source=dir_path+"/data/data_source/"

sub_path_data_source="demo_onefile/"

#sub_path_data_source="demo/"
#sub_path_data_source="small/"

file_path=path_data_source+sub_path_data_source

file_names = [os.path.join(file_path, f) 
                      for f in os.listdir(file_path) 
                      if f.endswith(".txt")]

#sub_path_data_source="demo/"





NameError: name '__file__' is not defined

### Table Database

In [4]:

#path_database=dir_path+"/data/database/csv/"
#path_sub_tfidf="sub_tfidf/"
#path_full_tfidf="full_tfidf/"
#path_tf="sub_tf/"
#path_database='/home/fsg/output_csv/database/'
#path_database='/media/fsg/74C86089C8604C04/PHD/Softwares/ph/database/' 

## Data Source

In [5]:
data_source = 'cs.txt'


## Tables

#### Table Data Base

In [6]:
#TF_File="TF-"
#TF_Full="TF-Full.csv"


### XML Data Base

# Configuration session

In [8]:
'''config = tf.ConfigProto(device_count={"CPU": 7},
                        allow_soft_placement=True,
                        inter_op_parallelism_threads=1,
                        intra_op_parallelism_threads=1,
                        use_per_session_threads=True)'''
#import pprint


#os.environ['TF_CPP_MIN_LOG_LEVEL']='5'
config = tf.ConfigProto(device_count={"GPU": 2,"CPU":1},
                        allow_soft_placement=True,
                        inter_op_parallelism_threads=16,
                        intra_op_parallelism_threads=16,
                        use_per_session_threads=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True

#pprint.pprint({ k:str(getattr(config,k)) for k in sorted(dir(config)) if not k.startswith('_')})


# Generic function

## Write data to csv

In [ ]:
'''
Write Excell sheet
'''
def save_file_to_database(data_rows,path_database,file_databbase,header_list):
    import csv
    outfile = open(path_database+file_databbase,'w')
    writer=csv.writer(outfile)
    #header_list=['uuid','paragraph','doc_id']
    i=0
    for line in data_rows:
        row=[i,line,'paragraph no.'+str(i)]
        if i==0:
            
            writer.writerow(header_list)
            writer.writerow(row)
        else:
            #print('ff')
            writer.writerow(row)
        i+= 1
        #outfile.close()
            

## Read data from csv

In [ ]:
'''
Read Excell sheet
'''
def read_text_from_database(path_database,file_databbase):
    import csv
    queue_paragraph=[]
    #f = open(sys.argv[1], 'rt')
    outfile = open(path_database+file_databbase,'rt')
    try:
                
        reader=csv.reader(outfile)
        for row in reader:
            queue_paragraph.append(row)
            #print (row)
    finally:
        print ("row")
        outfile.close()
        
    return queue_paragraph
    

In [ ]:
def read_cvs_by_pands(path_database,file_databbase,index_col, header):
    import csv
    return pd.read_csv(path_database+file_databbase,index_col=index_col,header=header)

#read_cvs_by_pands(path_database,paragraph_table,index_col=0,header=0)

In [ ]:
def write_cvs_by_pands(path_database,file_databbase,header,data_rows):
    import csv
    csv_df=pd.DataFrame(data_rows,columns=header ) 
    csv_df.to_csv(path_database+file_databbase)

    

#write_cvs_by_pands(path_database,sentences_paragraph_table,sentences_paragraph_list,sales)

In [ ]:
#save pragraphs to files
def write_file(pragraph,num_pragraph,path):
    file = open(path+str(num_pragraph)+".txt","w") 
 
    file.write(pragraph) 
    
    file.close() 
    

In [ ]:
#create sub dataset
def sub_dataset(path_data_source,data_source):
    pragraphs=txt_pragraphs(read_file(path_data_source+data_source))
    counter=0
    for pragraph in pragraphs:
        print('pragraph no ',counter)
        write_file(pragraph,counter,sub_path_data_source)
        counter +=1
    

# Pre processing pipeline

## Read Huge File

In [ ]:
def read_file(str):
    file = open(str,'r')
    txt=file.read()
    #print(txt)
    return txt

## Split Document to pragraphs

In [ ]:
def txt_pragraphs(str):
    pragraphs = str.split("\n\n")
    return pragraphs
#pragraphs=txt_pragraphs(txt)
#type(pragraphs)

## Split Paragraph to Sentences

In [ ]:
def pragraph_to_setnences(str):
    from nltk.tokenize import sent_tokenize, word_tokenize
    return sent_tokenize(str)
#setnences=pragraph_to_setnences(pragraphs[n_pragraph])

## Word Process For Sentence

### Removing English stopwords and Punct per Sentence

In [6]:
new_stop_words = ['the', 'that', 'to', 'as', 'there', 'has', 'and', 'or', 'is', 'not', 'a', 'of', 'but', 'in', 'by', 'on', 'are', 'it', 'if','what','where','how','when']
new_stop_words2=['--','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now','even','until','then','must']
numbers=[1,2,3,4,5,6,7,8,9]
#stemmer = SnowballStemmer("english", ignore_stopwords=True)
def remove_stopword_sentences(sent):
    import nltk
    from nltk.corpus import wordnet as wn
    import time
    from nltk.corpus import stopwords
    from nltk.tokenize import RegexpTokenizer
    from string import digits
    from nltk.corpus.reader.wordnet import WordNetError
    import sys
    list_word=[]

    try:
        tokenizer = RegexpTokenizer("[\w']+")
    
        words=tokenizer.tokenize(sent)
    
        english_stops = set(stopwords.words('english'))
        #stems=[]
        
        list_word=[word for word in words if word.lower() not in english_stops and word.lower() not in new_stop_words and word.lower() not in new_stop_words2 and  not word.lower().isdigit() and word.lower() not in digits and word.lower() not in  numbers]
    
    #for word in list_word:
        #stems.append(stem(word))
        #stems.append(PorterStemmer().stem(word))
        #stems.append(stemmer.stem(word))
        #stems.append(stemmer.stem("computer"))
        #stems.append(word)
    except WordNetError as e:
        print("WordNetError on concept {}".format(sent))
    except AttributeError as e:
        print("Attribute error on concept {}: {}".format(sent, e))
    except:
        print("Unexpected error on concept {}: {}".format(sent, sys.exc_info()[0]))
    
    return list_word#stems#(stem(setem_word for setem_word in  ([word for word in words if word not in english_stops and word not in new_stop_words])))

In [ ]:
def word_list_sentece(pragraph):
    words_list=[]
    setnences=pragraph_to_setnences(pragraph)
    for indexs in range(len(setnences)):    
        #print("Sentence No. ",indexs,": ",setnences[indexs],"\n")
        words=remove_stopword_sentences(setnences[indexs])
        wordsent=''
        for index in range(len(words)):
            wordsent+=' '+words[index]
            #print("wordsent:",wordsent)
            
        words_list.append(wordsent)
        #count = Counter(words)
        #print("wordsent:",wordsent)
        #print(" word:",words)
    print(words_list)
    return words_list

#corpus=word_list_sentece(pragraphs[0])

## Word Sense Disambiguation (WSD): LESK per Sentence

Given an ambiguous word and the context in which the word occurs, Lesk returns a Synset with the highest number of overlapping words between the context sentence and different definitions from each Synset.


In [ ]:

'''
this function for compute lesk for each word(list of word) in sentence
'''
def lesk_words_sentence(words,sentence):
    import nltk
    from nltk.corpus import wordnet as wn
    from nltk.wsd import lesk
    lesks= []
    for word in words:
        if lesk(sentence,word, 'n') is not None:
            lesks.append(lesk(sentence,word, 'n'))
            #print("Word is: ",word,"\n LESK: ",lesk(sentence,word, 'n'),"\n Sentence: ",sentence )
        
    return lesks

In [48]:
'''
this function for compute lesk of word in sentence
'''

def lesk_word_sentence(sentence,word):
    import nltk
    from nltk.corpus import wordnet as wn
    from nltk.wsd import lesk
    from nltk.corpus.reader.wordnet import WordNetError
    import sys
    disambiguated=''
    #print(type(disambiguated))
    try:
        
    #lesks= []
    #for word in words:
    #disambiguated=lesk(context_sentence=sentence, ambiguous_word=word)
    
        disambiguated=lesk(sentence,word, 'n')
        print(type(disambiguated))
    #print(disambiguated)
    #if disambiguated is not None:
        #lesk_synset=disambiguated
    #else:
    #lesk_synset=0
    #print("Word is: ",word,"\n LESK: ",lesk(sentence,word, 'n'),"\n Sentence: ",sentence )
    except WordNetError as e:
        print("WordNetError on concept {}".format(sentence))
    except AttributeError as e:
        print("Attribute error on concept {}: {}".format(sentence, e))
    except:
        print("Unexpected error on concept {}: {}".format(sentence, sys.exc_info()[0]))    
    return disambiguated

#lesk("Computer science is a discipline that spans theory and practice","science")

#sent = 'people should be able to marry a person of their choice'.split()
#lesk(sent, 'able')

In [51]:

type(lesk_word_sentence("I love dog","dog"))

<class 'str'>
<class 'nltk.corpus.reader.wordnet.Synset'>


nltk.corpus.reader.wordnet.Synset

In [33]:
def add_row_csv(path_database,idf,list_data):
    import csv
    with open(path_database+idf, 'a') as f:
        writer = csv.writer(f)
        writer.writerow(list_data)

In [ ]:
def name_file(full_name_path):
    d=full_name_path.split("/")
    print(d)
    name=d[len(d)-1].split(".")
    return name[0]

In [82]:
#with tf.control_dependencies(paragraph_list):
def MyLoop(gpuname,file_list_task,no_thread):
    import csv
    
    import nltk
    from nltk.wsd import lesk
    from nltk.corpus import wordnet as wn
    from collections import Counter
    from nltk.corpus.reader.wordnet import WordNetError
    import sys

    import pandas as pd
    import numpy as np
    from nltk.tokenize import sent_tokenize, word_tokenize

    from nltk.corpus import stopwords
    from nltk.tokenize import RegexpTokenizer
    from nltk import PorterStemmer
    #from nltk.stem.porter import *
    from nltk.stem.snowball import SnowballStemmer
    from string import digits
    with tf.device(gpuname):
        
        path_database=dir_path+"/data/database/csv/"
        path_sub_tfidf="sub_tfidf/"
        path_full_tfidf="full_tfidf/"
        path_tf="sub_tf/"
        TF_File="TF-"
        TF_Full="TF-Full.csv"
                
        for i in range(len(file_list_task)):    
            #with tf.Session(config=config) as sess:
            #index_paragraph=0
            col=1

            index_file=0
            #sess.run(tf.global_variables_initializer())
            #print(file_names)

            #for filename in file_names:
                #print("index_file",str(index_file))
            word_file_fatma=[]
            filename=file_list_task[i]
            with open(filename) as inf:
                    #print("tpe",type(inf))
                txt=inf.read()

                paragraph_list=txt_pragraphs(txt)   


                for paragraph in paragraph_list: #get pragraphs(documents) from DB
                        #print("Pragraph type ",type(paragraph))


                    #if index_paragraph ==0:
                        #index_paragraph += 1
                    #else:

                    setnences=pragraph_to_setnences(paragraph)#partitions paragraph to sentence


                    for setnence in setnences:
                                #print("  ",setnence)                            

                        words=remove_stopword_sentences(setnence)#remove stop words and noise
                        try:

                            for word in words:




                                lesk=lesk_word_sentence(setnence,word)#get LESK of word in sentence


                                    #paragraph_word.append(word_sentence)

                                if lesk is not None:
                                    #print("type of lesk in words",type(lesk),lesk)

                                    word_file_fatma.append(lesk.name())

                        except WordNetError as e:
                                print("WordNetError on concept {}".format(e))
                        except AttributeError as e:
                                print("Attribute error on concept {}".format(e))
                        except:
                                print("Unexpected error on concept {}".format(sys.exc_info()[0]))



                        '''////////////////END Sentence////////////////# '''


                    #write_cvs_by_pands(path_database,word_sentences_table,word_sentences_list,word_sentences_list_data)


                    '''////////////////END PARAGRAPH////////////////# '''

            #write_cvs_by_pands(path_database,sentences_paragraph_table,sentences_paragraph_list,sentences_paragraph_list_data)

            #print(word_file_fatma)

            word_file_Freq=Counter(word_file_fatma)
            sum_count=sum(word_file_Freq.values())

            #print(type(word_file_Freq))
            #print(word_file_Freq)
            #csv_df=pd.DataFrame([word_file_Freq],columns=word_file_Freq.keys() ) 
            freq=[]
            for i in word_file_Freq.values():
                c=i/sum_count
                freq.append(c)
            csv_df=pd.DataFrame([freq],columns=word_file_Freq.keys() ) 


            csv_df.to_csv(path_database+path_tf+TF_File+name_file(filename)+".csv")
                # add to idf file 

            full_list=[]
            full_list.insert(0,name_file(filename))
            full_list=full_list+list(word_file_Freq.keys())
            # add to single

            add_row_csv(path_database+path_sub_tfidf,name_file(filename)+".csv",full_list)
            # add to total idf file 
            add_row_csv(path_database+path_full_tfidf,TF_Full,full_list)
            index_file +=1



NameError: name 'file_names' is not defined

In [10]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
    #return [x.name for x in local_device_protos]


In [11]:
def test(gpuname,file_list_task,no_thread):
    print("\n Thread No:",no_thread,"No File",len(file_list_task),"\n")
    for i in range(len(file_list_task)):
        with tf.device("/job:localhost/task:"+str(i)+"/device:"+gpuname[1:]):

            #for i in range(len(file_list_task)):
            print("/job:localhost/task:",i,"device:",gpuname[1:],"file-->",file_list_task[i],"\n")



In [ ]:
def test2(gpuname,file_list_task,no_thread):
    
    with tf.device(gpuname):
        print("\n Thread No:",no_thread,"No File",len(file_list_task),"Devices",gpuname,"\n")
        for i in range(len(file_list_task)):

            #for i in range(len(file_list_task)):
            print("/job:localhost/task:",i,"device:",gpuname[1:],"file-->",file_list_task[i],"\n")



In [16]:
import threading
from tensorflow.python.client import timeline
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    threads=[]
    index=1
    file_names=['cs1','cs2','cs3','cs4','cs5','cs6','cs7']
    #print("in session",len(file_names))
    
    gpu_list=['/gpu:0','/gpu:1']#get_available_gpus()
    
    sub_file_lsit_size=round(len(file_names)/len(gpu_list))
    #print(sub_file_lsit_no)
    total_file_list=[]
    index_file_name=0
   


    for i in range(len(gpu_list)):
        sub_file_list=[]
        '''
        to devide total file list on gpu
        '''
        for x in range(sub_file_lsit_size):
            if index_file_name < len(file_names):
                sub_file_list.append(file_names[index_file_name])
                index_file_name +=1
                #print(index_file_name)
            
        total_file_list.append(sub_file_list)
    coord = tf.train.Coordinator()    
    for i in range(len(gpu_list)):
        
        #thread = threading.Thread(target=test, args=(gpu_list[i],total_file_list[i],i))
        #threads.append(thread)
        #with tf.device(gpu_list[i]):
        #test(gpu_list[i],total_file_list[i],i)
    
    
    
    
    
    
    #for t in threads: t.start()
    #coord.join(threads)
    #print(total_file_list)
    
    
            
            
        
    
    
    '''for filename in file_names:
        # Coordinate the loading of image files.
        coord = tf.train.Coordinator()
        thread = threading.Thread(target=MyLoop, args=(coord,sess,filename))
        # thread.start()
        threads.append(thread)
        print("Thread No.",index)
        index +=1



    for t in threads: t.start()
    coord.join(threads)'''




NameError: name 'gpuname' is not defined

In [17]:
import tensorflow as tf
c = tf.constant("Hello, distributed TensorFlow!")
server = tf.train.Server.create_local_server()
sess = tf.Session(server.target)  # Create a session on the server.
sess.run(c)



b'Hello, distributed TensorFlow!'

In [ ]:
c = tf.constant("Hello, distributed TensorFlow!")
tf.train.ClusterSpec({
    "worker": [
        "localhost:2222"
              ],
    "ps": [
        "localhost:2222"
        
    ]})
# In task 0:
cluster = tf.train.ClusterSpec({"local": ["localhost:2222", "localhost:2223"]})
server = tf.train.Server(cluster, job_name="local", task_index=0)
server.join()
sess = tf.Session(server.target)  # Create a session on the server.
sess.run(c)


In [8]:
# try running cpu intensive test on two devices
print("start")
import tensorflow as tf
import time

def matmul_op():
    """Multiply two matrices together"""
  
    n = 2000
    a = tf.ones((n, n), dtype=tf.float32)
    c=tf.matmul(a, a)/n
    print("c",c)
    for i in range(10):
        print(i)
    return c #tf.matmul(a, a)/n

slow_op  = matmul_op

with tf.device("/cpu:0"):
    one = slow_op()
with tf.device("/cpu:1"):
    another_one = slow_op()

config = tf.ConfigProto(device_count={"CPU": 2},
                        inter_op_parallelism_threads=2,
                        intra_op_parallelism_threads=1)
config.graph_options.optimizer_options.opt_level = -1

sess = tf.Session(config=config)
#two = one+another_one

# pre-warm the kernels
#print("ccc",sess.run(one))

start  = time.time()
print("start one",start)
sess.run(one)
end=time.time()
print("end one",start)
elapsed_time = end - start
print("Single op: %2.4f sec"%(elapsed_time))

start  = time.time()
print("start two",start)
sess.run(two)
end=time.time()
print("end two",start)
elapsed_time2 = end-start
print("Two ops in parallel: %.2f sec (%.2f times slower)"%(elapsed_time2,
elapsed_time2/elapsed_time))

start
c Tensor("truediv_10:0", shape=(2000, 2000), dtype=float32, device=/device:CPU:0)
0
1
2
3
4
5
6
7
8
9
c Tensor("truediv_11:0", shape=(2000, 2000), dtype=float32, device=/device:CPU:1)
0
1
2
3
4
5
6
7
8
9
start one 1510852221.6026764
end one 1510852221.6026764
Single op: 0.9528 sec
start two 1510852222.5558467
end two 1510852222.5558467
Two ops in parallel: 2.00 sec (2.10 times slower)


In [10]:
s = '/gpu:0'
n = 1
a, s = s[:n], s[n:]
print(a,"",s)

/  gpu:0


In [13]:
x=[]

for i in range(1000000):
    s="cs "+str(i)
    
    x.append(s)
    

In [14]:
x

['cs 0',
 'cs 1',
 'cs 2',
 'cs 3',
 'cs 4',
 'cs 5',
 'cs 6',
 'cs 7',
 'cs 8',
 'cs 9',
 'cs 10',
 'cs 11',
 'cs 12',
 'cs 13',
 'cs 14',
 'cs 15',
 'cs 16',
 'cs 17',
 'cs 18',
 'cs 19',
 'cs 20',
 'cs 21',
 'cs 22',
 'cs 23',
 'cs 24',
 'cs 25',
 'cs 26',
 'cs 27',
 'cs 28',
 'cs 29',
 'cs 30',
 'cs 31',
 'cs 32',
 'cs 33',
 'cs 34',
 'cs 35',
 'cs 36',
 'cs 37',
 'cs 38',
 'cs 39',
 'cs 40',
 'cs 41',
 'cs 42',
 'cs 43',
 'cs 44',
 'cs 45',
 'cs 46',
 'cs 47',
 'cs 48',
 'cs 49',
 'cs 50',
 'cs 51',
 'cs 52',
 'cs 53',
 'cs 54',
 'cs 55',
 'cs 56',
 'cs 57',
 'cs 58',
 'cs 59',
 'cs 60',
 'cs 61',
 'cs 62',
 'cs 63',
 'cs 64',
 'cs 65',
 'cs 66',
 'cs 67',
 'cs 68',
 'cs 69',
 'cs 70',
 'cs 71',
 'cs 72',
 'cs 73',
 'cs 74',
 'cs 75',
 'cs 76',
 'cs 77',
 'cs 78',
 'cs 79',
 'cs 80',
 'cs 81',
 'cs 82',
 'cs 83',
 'cs 84',
 'cs 85',
 'cs 86',
 'cs 87',
 'cs 88',
 'cs 89',
 'cs 90',
 'cs 91',
 'cs 92',
 'cs 93',
 'cs 94',
 'cs 95',
 'cs 96',
 'cs 97',
 'cs 98',
 'cs 99',
 'cs 100',